### Description 

1.  Download dataset from https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/
2.  Explore dataset:
       - Feature analysis(distributions, correlation)
       - Target analysis(balance/imbalance)
3.  Use algorithms from trees and linear notebooks: 
       - tune model parameters : use GridSearch, KFold
       - try different metrics : accuracy, roc auc
       - algorithm visualization
       - algorithm comparing     
4. Advanced: try [XGBoost](https://xgboost.readthedocs.io/en/latest/), [LightGBM](https://lightgbm.readthedocs.io/en/latest/) or [CatBoost](https://catboost.ai/) 
5. Conclusion


In [2]:
# import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib

%matplotlib inline

In [6]:
# load data
data = pd.read_csv('spambase.zip')


ValueError: ('Multiple files found in compressed zip file %s', "['spambase.DOCUMENTATION', 'spambase.data', 'spambase.names']")